<a href="https://colab.research.google.com/github/xia0gui/YOLOv8-Classification-Pneumothorax/blob/main/2_datapreprocessing_augmenting_datapathformatsetup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Library**

In [1]:
# -*- coding: utf-8 -*-
# Google Colab Setup and Library Imports
from google.colab import drive
import os
import glob
import numpy as np
from PIL import Image
import cv2
import keras
import matplotlib.pyplot as plt
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Data Preprocessing**

## **Augmenting**

In [ ]:
import os
import cv2
import albumentations as A

# Define the paths to image folders
no_pneumothorax_dir = '/content/drive/MyDrive/kaggle_dataset/selected_class_0'
pneumothorax_dir = '/content/drive/MyDrive/kaggle_dataset/class_1'

# Define output directories for augmented images
augmented_no_pneumothorax_output_path = '/content/drive/MyDrive/kaggle_dataset/augmented_no_pneumothorax'
augmented_pneumothorax_output_path = '/content/drive/MyDrive/kaggle_dataset/augmented_pneumothorax'
os.makedirs(augmented_no_pneumothorax_output_path, exist_ok=True)
os.makedirs(augmented_pneumothorax_output_path, exist_ok=True)

# Function to apply CLAHE
def apply_clahe(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(image)

# Function to save an augmented image
def save_augmented_image(image, image_name, output_img_dir):
    aug_image_path = os.path.join(output_img_dir, f"{image_name}_aug.png")
    cv2.imwrite(aug_image_path, image)

# Define the augmentation pipeline using Albumentations
augmentations = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.0, p=0.5),  # Adjust brightness and contrast
    A.ShiftScaleRotate(shift_limit=0.0, scale_limit=0.1, rotate_limit=0, p=0.5),  # No shift, got scale, and no rotation
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.2)  # Add Gaussian noise
])

# Function to augment images
def augment_images(image_paths, augmentations, output_img_dir):
    total_images = len(image_paths)

    for i, image_path in enumerate(image_paths):
        # Load the original image using OpenCV and apply CLAHE
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = apply_clahe(image)

        image_name = os.path.splitext(os.path.basename(image_path))[0]

        # Apply augmentations to the CLAHE-enhanced image
        augmented = augmentations(image=image)
        aug_image = augmented['image']

        # Save the augmented image
        save_augmented_image(aug_image, image_name, output_img_dir)

        print(f"Processed {i+1}/{total_images} images for augmentation.")

# Get all image paths in the train and validation overlayed directories
no_pneumothorax_image_paths = [os.path.join(no_pneumothorax_dir, img) for img in os.listdir(no_pneumothorax_dir)]
pneumothorax_image_paths = [os.path.join(pneumothorax_dir, img) for img in os.listdir(pneumothorax_dir)]

# Augment and save images to respective output directories
augment_images(no_pneumothorax_image_paths, augmentations, augmented_no_pneumothorax_output_path)
augment_images(pneumothorax_image_paths, augmentations, augmented_pneumothorax_output_path)


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Streaming output truncated to the last 5000 lines.
Processed 339/2669 images for augmentation.
Processed 340/2669 images for augmentation.
Processed 341/2669 images for augmentation.
Processed 342/2669 images for augmentation.
Processed 343/2669 images for augmentation.
Processed 344/2669 images for augmentation.
Processed 345/2669 images for augmentation.
Processed 346/2669 images for augmentation.
Processed 347/2669 images for augmentation.
Processed 348/2669 images for augmentation.
Processed 349/2669 images for augmentation.
Processed 350/2669 images for augmentation.
Processed 351/2669 images for augmentation.
Processed 352/2669 images for augmentation.
Processed 353/2669 images for augmentation.
Processed 354/2669 images for augmentation.
Processed 355/2669 images for augmentation.
Processed 356/2669 images for augmentation.
Processed 357/2669 images for augmentation.
Processed 358/2669 images for augmentation.
Processed 359/2669 images for augmentation.
Processed 360/2669 images

In [2]:
import os

# Define the directory
selected_images_dir = '/content/drive/MyDrive/kaggle_dataset/augmented_no_pneumothorax'

# Count the number of image files
image_count = len([f for f in os.listdir(selected_images_dir) if f.endswith('.png')])

# Print the result
print(f"Total number of images in '{selected_images_dir}': {image_count}")


Total number of images in '/content/drive/MyDrive/kaggle_dataset/augmented_no_pneumothorax': 2669


In [ ]:
import os

# Define the directory
selected_images_dir = '/content/drive/MyDrive/kaggle_dataset/augmented_pneumothorax'

# Count the number of image files
image_count = len([f for f in os.listdir(selected_images_dir) if f.endswith('.png')])

# Print the result
print(f"Total number of images in '{selected_images_dir}': {image_count}")


Total number of images in '/content/drive/MyDrive/kaggle_dataset/augmented_pneumothorax': 2669


## **Setting up Dataset Format**

In [ ]:
import os
import shutil
import random
from sklearn.model_selection import KFold
from tqdm import tqdm  # For the progress bar

# Paths to the datasets
no_pneumothorax_path = '/content/drive/MyDrive/kaggle_dataset/augmented_no_pneumothorax'
pneumothorax_path = '/content/drive/MyDrive/kaggle_dataset/augmented_pneumothorax'

# Output directory for the 5 folds
output_dir = '/content/drive/MyDrive/kaggle_dataset/kfold_training_data'
os.makedirs(output_dir, exist_ok=True)

# Get list of images in each category
no_pneumothorax_images = [os.path.join(no_pneumothorax_path, img) for img in os.listdir(no_pneumothorax_path) if img.endswith(('.png', '.jpg', '.jpeg'))]
pneumothorax_images = [os.path.join(pneumothorax_path, img) for img in os.listdir(pneumothorax_path) if img.endswith(('.png', '.jpg', '.jpeg'))]

# Combine all images with class labels
no_pneumothorax_labeled = [(img, 'no_pneumothorax') for img in no_pneumothorax_images]
pneumothorax_labeled = [(img, 'pneumothorax') for img in pneumothorax_images]

# Shuffle the images to ensure randomness
random.seed(42)  # For reproducibility
random.shuffle(no_pneumothorax_labeled)
random.shuffle(pneumothorax_labeled)

# Ensure both classes have the same number of splits
min_class_size = min(len(no_pneumothorax_labeled), len(pneumothorax_labeled))

# Use only the minimum number of samples from each class
no_pneumothorax_labeled = no_pneumothorax_labeled[:min_class_size]
pneumothorax_labeled = pneumothorax_labeled[:min_class_size]

# Create 5-fold splits for each class
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Function to create directories and copy files for each fold with a progress bar
def create_fold(fold_index, no_p_images, pneu_images):
    fold_folder = os.path.join(output_dir, f'fold_{fold_index + 1}')

    # Define class directories for this fold
    no_pneumothorax_fold_path = os.path.join(fold_folder, 'no_pneumothorax')
    pneumothorax_fold_path = os.path.join(fold_folder, 'pneumothorax')

    os.makedirs(no_pneumothorax_fold_path, exist_ok=True)
    os.makedirs(pneumothorax_fold_path, exist_ok=True)

    # Copy no_pneumothorax images to the fold with a progress bar
    print(f'\nCopying no_pneumothorax images for Fold {fold_index + 1}...')
    for img_path in tqdm(no_p_images, desc=f'Fold {fold_index + 1} - no_pneumothorax'):
        shutil.copy(img_path, os.path.join(no_pneumothorax_fold_path, os.path.basename(img_path)))

    # Copy pneumothorax images to the fold with a progress bar
    print(f'Copying pneumothorax images for Fold {fold_index + 1}...')
    for img_path in tqdm(pneu_images, desc=f'Fold {fold_index + 1} - pneumothorax'):
        shutil.copy(img_path, os.path.join(pneumothorax_fold_path, os.path.basename(img_path)))

# Generate folds for both classes
for fold_index, ((no_p_train_indices, no_p_val_indices), (pneu_train_indices, pneu_val_indices)) in enumerate(zip(kf.split(no_pneumothorax_labeled), kf.split(pneumothorax_labeled))):
    print(f'\nCreating Fold {fold_index + 1}...')

    # Get validation images for this fold
    no_p_val_images = [no_pneumothorax_labeled[i][0] for i in no_p_val_indices]
    pneu_val_images = [pneumothorax_labeled[i][0] for i in pneu_val_indices]

    # Create the fold with the selected images
    create_fold(fold_index, no_p_val_images, pneu_val_images)

print("\nData splitting into 5 folds completed successfully!")



Creating Fold 1...

Copying no_pneumothorax images for Fold 1...


Fold 1 - no_pneumothorax: 100%|██████████| 534/534 [00:50<00:00, 10.60it/s] 


Copying pneumothorax images for Fold 1...


Fold 1 - pneumothorax: 100%|██████████| 534/534 [02:08<00:00,  4.16it/s]



Creating Fold 2...

Copying no_pneumothorax images for Fold 2...


...


Creating Fold 5...

Copying no_pneumothorax images for Fold 5...


Fold 5 - no_pneumothorax: 100%|██████████| 533/533 [00:05<00:00, 98.62it/s]


Copying pneumothorax images for Fold 5...


Fold 5 - pneumothorax: 100%|██████████| 533/533 [02:09<00:00,  4.12it/s]


Data splitting into 5 folds completed successfully!
